In [1]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"The current device is {device}")

The current device is cuda:0


In [2]:


# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.eval()
model = model.to(device)

model.conf = 0.5  # NMS confidence threshold
      
# Images
imgs = 'data/bus.jpg' #['']  # batch of images

# Inference
results = model(imgs)

print(results)
results.save()

Using cache found in /home/students/sxa1507/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-7-9 Python-3.6.12 torch-1.10.1+cu102 CUDA:0 (GeForce RTX 2060, 5935MiB)



Python 3.7.0 required by YOLOv5, but Python 3.6.12 is currently installed


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Saved 1 image to runs/detect/exp2


image 1/1: 1080x810 4 persons, 1 bus
Speed: 17.8ms pre-process, 8.3ms inference, 1.2ms NMS per image at shape (1, 3, 640, 480)



In [12]:
import cv2
imgr = cv2.imread('data/bus.jpg')
imger = cv2.cvtColor(imgr, cv2.COLOR_BGR2RGB)

model.conf = 0.7  # confidence threshold (0-1)
model.iou = 0.3  # NMS IoU threshold (0-1)
model.classes = [0]

results = model(imger)
#frame = results.numpy()
#cv2.imwrite('outer.jpg',frame)
bbox = results.pandas().xyxy[0]
#results.crop()
bbox

,xmin,ymin,xmax,ymax,confidence,class,name
0,671.787842,395.372101,810.000000,878.361389,0.896172,0,person
1,220.657059,408.141083,346.167328,867.381104,0.870248,0,person
2,49.250805,389.990540,248.078201,912.458740,0.851563,0,person


In [20]:
for _,row in bbox.iterrows():
    #print(row['xmin'],type(row['xmin']))
    cv2.rectangle(imgr,(int(row['xmin']),int(row['ymin'])),(int(row['xmax']),int(row['ymax'])), (255,0,0) , 2  )
cv2.imwrite('outer.jpg',imgr)

True

In [32]:
import time

model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.3  # NMS IoU threshold (0-1)
model.classes = [0]


vidcap = cv2.VideoCapture('data/MOT20-01-raw.webm')
ret, image_orig = vidcap.read()
count = 1
width  = int(vidcap.get(3))
height = int(vidcap.get(4))
#print('image shape:',image.shape)
infer_out = cv2.VideoWriter('output.mp4', 
                         cv2.VideoWriter_fourcc(*'mp4v'),
                         int(vidcap.get(cv2.CAP_PROP_FPS)), (width,height))
t1 = time.time()
while ret:
    #cv2.imshow("window",image)
    #cv2.waitKey(34)
    t2 = time.time()
    
    imger = cv2.cvtColor(image_orig, cv2.COLOR_BGR2RGB)
    t3 = time.time()
    results = model(imger)
    
    bbox = results.pandas().xyxy[0]
    
    #inference stats
    if count%50 == 0:
        print(count,' frame infer time:',time.time()-t3," pre-process time:",t3-t2 ) 
    
    # agent bbox and wriye to video
    
    for _,row in bbox.iterrows():
        cv2.rectangle(image_orig,(int(row['xmin']),int(row['ymin'])),(int(row['xmax']),int(row['ymax'])), (255,0,0) , 2  )
    infer_out.write(image_orig)
    if count%50 == 0:
        print('frame infer+I/o time:',time.time()-t2 ) 
    count += 1
    ret, image_orig = vidcap.read()
print('total processing time:',time.time()-t1,' for frames : ',count ) 
infer_out.release()
vidcap.release()
    
#cv2.destroyAllWindows()




50  frame infer time: 0.010748624801635742  pre-process time: 8.58306884765625e-05
frame infer+I/o time: 0.01633429527282715
100  frame infer time: 0.010743379592895508  pre-process time: 9.226799011230469e-05
frame infer+I/o time: 0.0153045654296875
150  frame infer time: 0.01058197021484375  pre-process time: 9.512901306152344e-05
frame infer+I/o time: 0.015503644943237305
200  frame infer time: 0.010657310485839844  pre-process time: 8.869171142578125e-05
frame infer+I/o time: 0.016314268112182617
250  frame infer time: 0.01061105728149414  pre-process time: 9.369850158691406e-05
frame infer+I/o time: 0.015547037124633789
300  frame infer time: 0.010802030563354492  pre-process time: 9.775161743164062e-05
frame infer+I/o time: 0.016347408294677734
350  frame infer time: 0.010786056518554688  pre-process time: 8.749961853027344e-05
frame infer+I/o time: 0.015564918518066406
400  frame infer time: 0.010770797729492188  pre-process time: 9.34600830078125e-05
frame infer+I/o time: 0.015